# VGG19 & ResNet50 Implementation for Pneumonia Detection

In [1]:
#Pre-requisites for VGG19

In [7]:
# Install pydicom for reading .DICOM files
#!pip install pydicom

In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import pydicom
import pylab
from skimage.transform import resize
import pathlib
import keras
from keras.applications.densenet import DenseNet121
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.applications import VGG19,ResNet50
from keras.layers import Dropout,Flatten

Using TensorFlow backend.


In [2]:
# Set the path for training images
TRAIN_IMAGES ='E:/Machine Learning/Great Learning/Projects/GL Capstone Project/GL Capstone Project/Dataset/rsna-pneumonia-detection-challenge/stage_2_train_images/'
Dataset = 'E:/Machine Learning/Great Learning/Projects/GL Capstone Project/GL Capstone Project/Dataset/rsna-pneumonia-detection-challenge'
weights = 'E:/Machine Learning/Great Learning/Projects/GL Capstone Project/GL Capstone Project/Code Base/VGG19/'
weights_res = 'E:/Machine Learning/Great Learning/Projects/GL Capstone Project/GL Capstone Project/Code Base/ResNet50/'


In [3]:
# Read the training CSV File and remove duplicates on Patient Id
filepath = (Dataset+'/stage_2_train_labels.csv')
Images_df = pd.read_csv(filepath)
Images_model_df = Images_df[['patientId','Target']]
Images_model_df=Images_model_df.drop_duplicates(subset='patientId')

In [4]:
# Sample the training images for initial experimentation
Images_sample_df = Images_model_df.sample(frac=1.0,random_state=42)

In [5]:
# Get the count
Images_sample_df['Target'].value_counts()

0    20672
1     6012
Name: Target, dtype: int64

In [6]:
def train_test_dict(Images_sample_df,test_size,random_state=42): 
   # Split into train and test validation datasets
    train_df, test_df = train_test_split(Images_sample_df, test_size=0.02, random_state=42, stratify=Images_sample_df[['Target']])
   # Convert to dictionary with patient-id as key and target as value
    train_dict=train_df.set_index('patientId')['Target'].to_dict()
    test_dict=test_df.set_index('patientId')['Target'].to_dict()
    return [train_dict,test_dict]

In [7]:
train_dict,test_dict = train_test_dict(Images_sample_df,0.05)

In [8]:
len(test_dict)

534

In [9]:
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)
#your_module = drive.CreateFile({'id':'185RGsTqpjcdKK2afnDs5LVxIZW6GTQZj'})
#your_module.GetContentFile('generator.py')

In [10]:
# Define Custom Generator Class to be used in Model Generator
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, path,batch_size=128, dim=(224,224), n_channels=3,
                 n_classes=1, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.path = path
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dcm_file_sample = (self.path +"/"+ ID +".dcm")
            dcm_data_sample = pydicom.filereader.dcmread(dcm_file_sample)
            image = dcm_data_sample.pixel_array
            image_array = np.stack([image] * 3, axis=2)
            image_array = image_array / 255.
            image_array = resize(image_array, (224, 224), mode= 'constant', anti_aliasing=True)
            X[i,] = image_array

            # Store class
            y[i] = self.labels[ID]

        return X,y

In [11]:
from keras.models import load_model

# VGG19

In [12]:
# Define the VGG19 model pre-loaded with imagenet weights with last layer set as false
input_shape = (224, 224, 3)
num_of_class=1
img_in = Input(input_shape)              
model = VGG19(include_top= False , 
                weights=weights+'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',    
                input_tensor= img_in, 
                input_shape= input_shape,
                pooling ='avg') 

# The pre-trained model has classification output for 14 categories and hence Dense layer is defined with layer 1
x = model.output  
predictions = Dense(1, activation="sigmoid", name="predictions")(x)    
model = Model(inputs=img_in, outputs=predictions)

In [13]:
# Print the model summary
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
# Create Train and Test generator
train_generator = DataGenerator(list(train_dict.keys()), train_dict,path=TRAIN_IMAGES,batch_size=32)
validation_generator = DataGenerator(list(test_dict.keys()), test_dict,path=TRAIN_IMAGES,batch_size=1)

In [15]:
#Define Custom Metrics Functions to be used in Keras Training
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
#Set Early stopping parameter and Reduce Learning rate on Plateau
callbacks_list = [EarlyStopping(monitor='val_loss',patience=5,),
                  ModelCheckpoint(filepath=weights_res+'my_model.h5',monitor='val_loss',save_best_only=True,),
                  ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,)]

In [17]:
# Set only the last layer as Trainable
def model_train_layers(model,layer_name):
    model.trainable = True
    set_trainable = False
    for layer in model.layers:
        
        if layer.name == layer_name:
            set_trainable = True
            #print(layer.name)
        if set_trainable:
             layer.trainable = True
        else:
             layer.trainable = False

In [29]:
model_train_layers(model,"predictions")

predictions


In [30]:
# Placeholder to load the weights from previous epochs
#model2.load_weights(weights+"my_model.h5")

In [31]:
# Compile with binary cross entropy loss
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['acc',f1_m,precision_m, recall_m])

In [32]:
# Run Fit Generator
history=model.fit_generator(generator=train_generator,
                    epochs=3,
                    validation_data=validation_generator,
                    callbacks=callbacks_list)

Epoch 1/3
817/817 [==============================] - 6573s 8s/step - loss: 0.4903 - acc: 0.7751 - f1_m: 0.0124 - precision_m: 0.0479 - recall_m: 0.0077 - val_loss: 0.0786 - val_acc: 0.7772 - val_f1_m: 0.0019 - val_precision_m: 0.0019 - val_recall_m: 0.0019
Epoch 2/3
817/817 [==============================] - 4617s 6s/step - loss: 0.4625 - acc: 0.7781 - f1_m: 0.0703 - precision_m: 0.2222 - recall_m: 0.0443 - val_loss: 0.0885 - val_acc: 0.7846 - val_f1_m: 0.0112 - val_precision_m: 0.0112 - val_recall_m: 0.0112
Epoch 3/3
817/817 [==============================] - 4769s 6s/step - loss: 0.4524 - acc: 0.7829 - f1_m: 0.1479 - precision_m: 0.3814 - recall_m: 0.0999 - val_loss: 0.2481 - val_acc: 0.7865 - val_f1_m: 0.0150 - val_precision_m: 0.0150 - val_recall_m: 0.0150


# ResNet50

In [12]:
# Define the ResNet50 model pre-loaded with imagenet weights with last layer set as false
input_shape = (224, 224, 3)
num_of_class=1
img_in = Input(input_shape)              
model = ResNet50(include_top= False , 
                weights='imagenet',    
                input_tensor= img_in, 
                input_shape= input_shape,
                pooling ='avg') 

# The pre-trained model has classification output for 14 categories and hence Dense layer is defined with layer 1
x = model.output  
predictions = Dense(1, activation="sigmoid", name="predictions")(x)    
model = Model(inputs=img_in, outputs=predictions)

94658560/94653016 [==============================] - 37s 0us/step


In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [18]:
model_train_layers(model,"predictions")

In [19]:
# Compile with binary cross entropy loss
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['acc',f1_m,precision_m, recall_m])

In [20]:
# Run Fit Generator
history=model.fit_generator(generator=train_generator,
                    epochs=3,
                    validation_data=validation_generator,
                    callbacks=callbacks_list)

Epoch 1/3
817/817 [==============================] - 5241s 6s/step - loss: 0.4213 - acc: 0.8020 - f1_m: 0.4304 - precision_m: 0.5840 - recall_m: 0.3830 - val_loss: 0.5596 - val_acc: 0.7753 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/3
817/817 [==============================] - 4783s 6s/step - loss: 0.3987 - acc: 0.8182 - f1_m: 0.5070 - precision_m: 0.6532 - recall_m: 0.4618 - val_loss: 0.6232 - val_acc: 0.7734 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/3
817/817 [==============================] - 4693s 6s/step - loss: 0.3930 - acc: 0.8223 - f1_m: 0.5164 - precision_m: 0.6637 - recall_m: 0.4747 - val_loss: 1.0815 - val_acc: 0.2247 - val_f1_m: 0.2247 - val_precision_m: 0.2247 - val_recall_m: 0.2247


# Conclusion

In [74]:
#Vgg19 has very low F1 score in training and very minimal value for Validation. This cannot be considered for our model
#ResNet50 provides a average F1 score which can be considered for further evaluation
